In [43]:
import pandas as pd
import networkx as nx
import copy
import itertools

In [75]:
def update_nodes(G, results_df):

    names = results_df["Name"]
    for name in names:
        if name not in list(G.nodes()):
            G.add_node(name)
    return G



def add_event_results(G, results_df, event_name):

    event_results = results_df[["Name", event_name]].dropna()
    rankings = list(event_results[event_name].unique())
    rankings.sort(reverse=True)

    for rank in rankings[:-1]:
        
        # GET SURFERS WHO FINISHED WITH RANK
        rank_batch = event_results[event_results[event_name] == rank]
        rank_surfers = list(rank_batch["Name"])
        
        # REMOVE SURFERS WITH RANK FROM RESULTS DATAFRAME
        event_results = event_results[~event_results["Name"].isin(rank_surfers)]
        remaining_surfers = list(event_results["Name"])

        # CREATE EDGES FOR SURFERS WITH RANK
        for rank_surfer in rank_surfers:
            for other_surfer in remaining_surfers:

                # CASE WHERE EDGE ALREADY EXISTS
                if G.has_edge(rank_surfer, other_surfer):
                    G[rank_surfer][other_surfer]["weight"] += 1

                # CASE WHERE EDGE DOES NOT EXIST
                else:
                    G.add_edge(rank_surfer, other_surfer, weight=1)

    return G



def visualize_graph(G):

    # SET LAYOUT
    pos = nx.spring_layout(G, k=1, iterations=20)

    # GET NODE INFO
    in_degrees = dict(G.in_degree())

    # SET NODE SIZING
    node_sizes = [in_degrees[node]*30 for node in G.nodes()]

    # DRAW NETWORK
    nx.draw(G,
        node_size=node_sizes,
        pos=pos
    )



def condense_edge(G, n1, n2):

    # GET NODE WEIGHTS
    try:
        n1_n2 = G.get_edge_data(n1, n2)["weight"]
        n2_n1 = G.get_edge_data(n2, n1)["weight"]
    except:
        return G

    # GET DIFF.
    diff = n1_n2 - n2_n1

    # REMOVE EDGES
    G.remove_edges_from([(n1, n2), (n2, n1)])

    # ADD NET EDGE
    if diff > 0:
        G.add_edge(n1, n2, weight=diff)
    elif diff < 0:
        G.add_edge(n2, n1, weight=-diff)

    return G



def condense_all_edges(G):

    # GET ALL NODES
    all_nodes = list(G.nodes())

    # GET NODE PAIRS
    all_pairs = list(itertools.combinations(all_nodes, 2))

    # ITERATE OVER PAIRS
    for pair in all_pairs:
        G = condense_edge(G, pair[0], pair[1])

    return G

## Create network for results from all years

In [77]:
# SET YEAR RANGE
first_year = 2010; last_year = 2019
years = [str(y) for y in range(first_year, last_year+1)]

# INITIALIZE GRAPH
results_net = nx.DiGraph()

# LOOP OVER YEARS
for year in years:

    # LOAD DATA
    results_df = pd.read_csv(f"../data/results_data/wsl_results_{year}.csv")

    # UPDATE NODES
    update_nodes(results_net, results_df)

    # ADD RESULTS FOR ALL EVENTS
    events = list(results_df.drop("Name", axis=1).columns)
    for event in events:
        results_net = add_event_results(results_net, results_df, event)

# CONDENSE EDGES USING NET WEIGHTS
results_net = condense_all_edges(results_net)

In [79]:
# SAVE NETWORK
nx.write_gml(results_net, "../data/networks/overall_results_network.gml")